In [44]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [45]:
# Yelp url for restaurants in Morris County, NJ to be scraped
url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Morris+County%2C+NJ&ns=1'
response = requests.get(url)

In [46]:
# intialize variables and create empty lists
nextpage_link = True
restaurant_name = []
yelp_url = []
restaurant_rating = []
price_cuisine = []
 
while (nextpage_link):
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # class for restaurant details
    containers = soup.find_all('div', class_='scrollablePhotos__09f24__1PpB8')[1:]
    
    # scroll through each container
    for item in containers:
        #print(item)
        # find and add restaurant name to list
        rest_name = item.find('a',class_='link__09f24__1kwXV')
        restaurant_name.append(rest_name.text)
        
        # find and add restaurant url to list
        yelp_url.append("https://www.yelp.com" + rest_name['href'])
        
        # find and add rating (if exists) to list
        rating = item.find('div',class_='i-stars__09f24__1T6rz')
        if rating:
            restaurant_rating.append(rating['aria-label'].split(' ')[0])
        else:
            restaurant_rating.append("")
        
        # find and add cuisine to list
        price_cuisine.append(item.find('div',class_='priceCategory__09f24__2IbAM').text)

    # find the next page url
    next_page = soup.find('a', class_='next-link')
    
    # if there is a another page, assign the url, otherwise exit
    if next_page:        
        nextpage_url = next_page['href']
        response = requests.get(nextpage_url)
        #time.sleep(2)
    else:
        nextpage_link = False

# Create Dataframe
yelp_df = pd.DataFrame({'restaurant_name': restaurant_name,
                       'yelp_url': yelp_url,
                       'restaurant_rating': restaurant_rating,
                       'price_cuisine': price_cuisine})

yelp_df    

,restaurant_name,yelp_url,restaurant_rating,price_cuisine
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces"
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4,"$$American (New), Bars, Pizza"
2,El Lechon De Negron,https://www.yelp.com/biz/el-lechon-de-negron-d...,4,"Puerto Rican, Empanadas"
3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4,"$$Diners, Italian"
4,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian"
...,...,...,...,...
235,Denville Diner,https://www.yelp.com/biz/denville-diner-denvil...,4,$Diners
236,The Second Half On Main,https://www.yelp.com/biz/the-second-half-on-ma...,3.5,"$$American (Traditional), Bars"
237,Casa D Pizza,https://www.yelp.com/biz/casa-d-pizza-denville...,4,"$$Pizza, Italian"
238,The Tuscan Table,https://www.yelp.com/biz/the-tuscan-table-denv...,4,$$Italian


In [47]:
# create empty lists to hold address
address1 = []
address2 = []
restaurant_website = []

executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser =  Browser("chrome", **executable_path, headless=False)

# iterate through all the rows in the yelp dataframe
for index, row in yelp_df.iterrows():
    
    # visit each restaurant's url
    browser.visit(row['yelp_url'])

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # find the address tag
    address = soup.find('address').find_all('span', class_='raw__373c0__3rcx7')
    
    # find address1 and add to list
    street_address = address[0].text
    for i in range(1, len(address)-1):
        street_address = street_address + address[i].text + " "
    address1.append(street_address)
    
    # find the rest of the address
    address2 = address[len(address)-1].text
    
    rest_website = soup.find('div', class_='stickySidebar__373c0__3PY1o').find('a', class_='lemon--a__373c0__IEZFH').text

    if rest_website == "Get Directions":
        restaurant_website.append("")
    else:
        restaurant_website.append(rest_website)
    
browser.quit()

# add address columns to yelp dataframe
yelp_df['address1'] = address1
yelp_df['address2'] = address2
yelp_df['restaurant_website'] = restaurant_website
                  
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Chester Township, NJ 07930",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4,"$$American (New), Bars, Pizza",500 NJ-10,"Chester Township, NJ 07930",sbknj.com
2,El Lechon De Negron,https://www.yelp.com/biz/el-lechon-de-negron-d...,4,"Puerto Rican, Empanadas",23 E Main St,"Chester Township, NJ 07930",
3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4,"$$Diners, Italian",477 NJ-10,"Chester Township, NJ 07930",thecornerbistronj.com
4,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Chester Township, NJ 07930",4seasonswharton.com
...,...,...,...,...,...,...,...
235,Denville Diner,https://www.yelp.com/biz/denville-diner-denvil...,4,$Diners,17 Broadway,"Chester Township, NJ 07930",denvillediner.com
236,The Second Half On Main,https://www.yelp.com/biz/the-second-half-on-ma...,3.5,"$$American (Traditional), Bars",5 E Main StSte 15,"Chester Township, NJ 07930",secondhalfonmain.com
237,Casa D Pizza,https://www.yelp.com/biz/casa-d-pizza-denville...,4,"$$Pizza, Italian",276 E Main St,"Chester Township, NJ 07930",casadpizzadenville.com
238,The Tuscan Table,https://www.yelp.com/biz/the-tuscan-table-denv...,4,$$Italian,28 Diamond Spring Rd,"Chester Township, NJ 07930",thetuscantabledenville.com


In [48]:
# save to csv file
yelp_df.to_csv('../Resources/yelp_data.csv')